<a href="https://colab.research.google.com/github/CodeT1m/agrik_obj_detect/blob/master/AgrikAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
print("Hello colab")

Hello colab


In [ ]:
!pwd

/content


In [ ]:
!git --version

git version 2.17.1


In [29]:
# If you forked the repository, you can replace the link.
repo_url = 'https://github.com/CodeT1m/agrik_obj_detect'

# Number of training steps.
num_steps = 4000000  # 200000

# Number of evaluation steps.
num_eval_steps = 50

MODELS_CONFIG = {
    'ssd_mobilenet_v2': {
        'model_name': 'ssd_mobilenet_v2_coco_2018_03_29',
        'pipeline_file': 'ssd_mobilenet_v2_coco.config',
        'batch_size': 12
    },
    'faster_rcnn_inception_v2': {
        'model_name': 'faster_rcnn_inception_v2_coco_2018_01_28',
        'pipeline_file': 'faster_rcnn_inception_v2_pets.config',
        'batch_size': 12
    },
    'rfcn_resnet101': {
        'model_name': 'rfcn_resnet101_coco_2018_01_28',
        'pipeline_file': 'rfcn_resnet101_pets.config',
        'batch_size': 8
    }
}

# Pick the model you want to use
# Select a model in `MODELS_CONFIG`.
selected_model = 'ssd_mobilenet_v2'

# Name of the object detection model to use.
MODEL = MODELS_CONFIG[selected_model]['model_name']

# Name of the pipline file in tensorflow object detection API.
pipeline_file = MODELS_CONFIG[selected_model]['pipeline_file']

# Training batch size fits in Colabe's Tesla K80 GPU memory for selected model.
batch_size = MODELS_CONFIG[selected_model]['batch_size']

print(repo_url)

https://github.com/CodeT1m/agrik_obj_detect


# Clone the object_detection_demo repository or fork 

In [3]:
import os

%cd /content
print("ok")
repo_dir_path = os.path.abspath(os.path.join('.', os.path.basename(repo_url)))
print(repo_dir_path)
!git clone {repo_url}
%cd {repo_dir_path}
!git pull

/content
ok
/content/agrik_obj_detect
Cloning into 'agrik_obj_detect'...
remote: Enumerating objects: 265, done.
remote: Counting objects: 100% (265/265), done.
remote: Compressing objects: 100% (96/96), done.
remote: Total 70279 (delta 199), reused 206 (delta 151), pack-reused 70014
Receiving objects: 100% (70279/70279), 2.11 GiB | 23.94 MiB/s, done.
Resolving deltas: 100% (23561/23561), done.
Checking out files: 100% (46188/46188), done.
/content/agrik_obj_detect
Already up to date.


#Install Required packages

In [2]:
!pip install tensorflow-gpu==1.15
import tensorflow as tf
print(tf.__version__)

device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

     |████████████████████████████████| 411.5MB 41kB/s 
     |████████████████████████████████| 512kB 29.4MB/s 
     |████████████████████████████████| 3.8MB 46.8MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp36-none-any.whl size=7540 sha256=a2557026710d6d88ece4fe8a1b85e522b444bca3f80ace0e8602c8ea0b55e43e
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow 2.2.0 has requirement gast==0.3.3, but you'll have gast 0.2.2 which is incompatible.
ERROR: tensorflow 2.2.0 has requirement tensorboard<2.3.0,>=2.2.0, but you'll have tensorboard 1.15.0 which is incompatible.
ERROR: tensorflow 2.2.0 has requirement tensorflow-estimator<2.3.0,>=2.2.0, but you'll have tensorflow-estimator 1.15.1 which is incompatible.
ERROR: tensorflow-probability 0.10.0 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
  Found existing installation: tensorflow-estimator 2.2.0
    Uninst

In [4]:
from google.colab import drive
drive.mount('/content/drive')
!ln -s "/content/drive/My Drive" "/content/fypdrive"

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
import os

print(os.environ['PYTHONPATH'])

:/content/drive/My\ Drive/models/research/:/content/drive/My\ Drive/models/research/slim/:/content/drive/My\ Drive/models/research/:/content/drive/My\ Drive/models/research/slim/


In [5]:
%cd /content/fypdrive
!git clone --quiet https://github.com/tensorflow/models.git

!apt-get install -qq protobuf-compiler python-pil python-lxml python-tk

!pip install -q Cython contextlib2 pillow lxml matplotlib

!pip install -q pycocotools

!pip install tf-slim

%cd /content/drive/My\ Drive/models/research
!protoc object_detection/protos/*.proto --python_out=.

import os
#%cd /content/drive
os.environ['PYTHONPATH'] += ':/content/fypdrive/models:/content/fypdrive/models/research/:/content/fypdrive/models/research/slim/'

!python object_detection/builders/model_builder_test.py

/content/drive/My Drive
fatal: destination path 'models' already exists and is not an empty directory.
Selecting previously unselected package python-bs4.
(Reading database ... 144379 files and directories currently installed.)
Preparing to unpack .../0-python-bs4_4.6.0-1_all.deb ...
Unpacking python-bs4 (4.6.0-1) ...
Selecting previously unselected package python-pkg-resources.
Preparing to unpack .../1-python-pkg-resources_39.0.1-2_all.deb ...
Unpacking python-pkg-resources (39.0.1-2) ...
Selecting previously unselected package python-chardet.
Preparing to unpack .../2-python-chardet_3.0.4-1_all.deb ...
Unpacking python-chardet (3.0.4-1) ...
Selecting previously unselected package python-six.
Preparing to unpack .../3-python-six_1.11.0-2_all.deb ...
Unpacking python-six (1.11.0-2) ...
Selecting previously unselected package python-webencodings.
Preparing to unpack .../4-python-webencodings_0.5-2_all.deb ...
Unpacking python-webencodings (0.5-2) ...
Selecting previously unselected pac

In [ ]:
os.environ['PYTHONPATH']

':/content/drive/My\\ Drive/models/research/:/content/drive/My\\ Drive/models/research/slim/:/content/drive/My\\ Drive/models/research/:/content/drive/My\\ Drive/models/research/slim/:content/drive/My\\ Drive/models:/content/drive/My\\ Drive/models/research/:/content/drive/My\\ Drive/models/research/slim/:content/drive/My Drive/models:/content/drive/My Drive/models/research/:/content/drive/My Drive/models/research/slim/'

## Prepare `tfrecord` files

Use the following scripts to generate the `tfrecord` files.
```bash
# Convert train folder annotation xml files to a single csv file,
# generate the `label_map.pbtxt` file to `data/` directory as well.
python xml_to_csv.py -i data/images/train -o data/annotations/train_labels.csv -l data/annotations

# Convert test folder annotation xml files to a single csv.
python xml_to_csv.py -i data/images/test -o data/annotations/test_labels.csv

# Generate `train.record`
python generate_tfrecord.py --csv_input=data/annotations/train_labels.csv --output_path=data/annotations/train.record --img_path=data/images/train --label_map data/annotations/label_map.pbtxt

# Generate `test.record`
python generate_tfrecord.py --csv_input=data/annotations/test_labels.csv --output_path=data/annotations/test.record --img_path=data/images/test --label_map data/annotations/label_map.pbtxt
```

In [6]:
%cd {repo_dir_path}

# Convert train folder annotation xml files to a single csv file,
# generate the `label_map.pbtxt` file to `data/` directory as well.
!python xml_to_csv.py -i data/images/train -o data/annotations/train_labels.csv -l data/annotations

# Convert test folder annotation xml files to a single csv.
!python xml_to_csv.py -i data/images/test -o data/annotations/test_labels.csv

# Generate `train.record`
!python generate_tfrecord.py --csv_input=data/annotations/train_labels.csv --output_path=data/annotations/train.record --img_path=data/images/train --label_map data/annotations/label_map.pbtxt

# Generate `test.record`
!python generate_tfrecord.py --csv_input=data/annotations/test_labels.csv --output_path=data/annotations/test.record --img_path=data/images/test --label_map data/annotations/label_map.pbtxt

/content/agrik_obj_detect
Successfully converted xml to csv.
Generate `data/annotations/label_map.pbtxt`
Successfully converted xml to csv.


W0712 21:46:22.007187 140085318506368 module_wrapper.py:139] From generate_tfrecord.py:107: The name tf.python_io.TFRecordWriter is deprecated. Please use tf.io.TFRecordWriter instead.


W0712 21:46:27.347840 140085318506368 module_wrapper.py:139] From generate_tfrecord.py:53: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.

Successfully created the TFRecords: /content/agrik_obj_detect/data/annotations/train.record


W0712 21:46:49.154347 140518348449664 module_wrapper.py:139] From generate_tfrecord.py:107: The name tf.python_io.TFRecordWriter is deprecated. Please use tf.io.TFRecordWriter instead.


W0712 21:46:50.560872 140518348449664 module_wrapper.py:139] From generate_tfrecord.py:53: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.

Successfully created the TFRecords: /content/agrik_o

In [7]:
test_record_fname = '/content/agrik_obj_detect/data/annotations/test.record'
train_record_fname = '/content/agrik_obj_detect/data/annotations/train.record'
label_map_pbtxt_fname = '/content/agrik_obj_detect/data/annotations/label_map.pbtxt'

##Download base model

In [8]:
%cd /content/fypdrive/models/research

import os
import shutil
import glob
import urllib.request
import tarfile
MODEL_FILE = MODEL + '.tar.gz'
DOWNLOAD_BASE = 'http://download.tensorflow.org/models/object_detection/'
DEST_DIR = '/content/fypdrive/models/research/pretrained_model'

if not (os.path.exists(MODEL_FILE)):
    urllib.request.urlretrieve(DOWNLOAD_BASE + MODEL_FILE, MODEL_FILE)

tar = tarfile.open(MODEL_FILE)
tar.extractall()
tar.close()

os.remove(MODEL_FILE)
if (os.path.exists(DEST_DIR)):
    shutil.rmtree(DEST_DIR)
os.rename(MODEL, DEST_DIR)

/content/drive/My Drive/models/research


In [9]:
!echo {DEST_DIR}
!ls -alh {DEST_DIR}

/content/fypdrive/models/research/pretrained_model
total 135M
-rw------- 1 root root   77 Mar 30  2018 checkpoint
-rw------- 1 root root  67M Mar 30  2018 frozen_inference_graph.pb
-rw------- 1 root root  65M Mar 30  2018 model.ckpt.data-00000-of-00001
-rw------- 1 root root  15K Mar 30  2018 model.ckpt.index
-rw------- 1 root root 3.4M Mar 30  2018 model.ckpt.meta
-rw------- 1 root root 4.2K Mar 30  2018 pipeline.config
drwx------ 3 root root 4.0K Mar 30  2018 saved_model


In [10]:
fine_tune_checkpoint = os.path.join(DEST_DIR, "model.ckpt")
fine_tune_checkpoint

'/content/fypdrive/models/research/pretrained_model/model.ckpt'

#Configuring a Training Pipeline

In [30]:
import os
pipeline_fname = os.path.join('/content/fypdrive/models/research/object_detection/samples/configs/', pipeline_file)

assert os.path.isfile(pipeline_fname), '`{}` not exist'.format(pipeline_fname)

In [31]:
def get_num_classes(pbtxt_fname):
    from object_detection.utils import label_map_util
    label_map = label_map_util.load_labelmap(pbtxt_fname)
    categories = label_map_util.convert_label_map_to_categories(
        label_map, max_num_classes=35, use_display_name=True)
    category_index = label_map_util.create_category_index(categories)
    return len(category_index.keys())

In [32]:
import re

num_classes = get_num_classes(label_map_pbtxt_fname)
with open(pipeline_fname) as f:
    s = f.read()
with open(pipeline_fname, 'w') as f:
    
    # fine_tune_checkpoint
    s = re.sub('fine_tune_checkpoint: ".*?"',
               'fine_tune_checkpoint: "{}"'.format(fine_tune_checkpoint), s)
    
    # tfrecord files train and test.
    s = re.sub(
        '(input_path: ".*?)(train.record)(.*?")', 'input_path: "{}"'.format(train_record_fname), s)
    s = re.sub(
        '(input_path: ".*?)(val.record)(.*?")', 'input_path: "{}"'.format(test_record_fname), s)

    # label_map_path
    s = re.sub(
        'label_map_path: ".*?"', 'label_map_path: "{}"'.format(label_map_pbtxt_fname), s)

    # Set training batch_size.
    s = re.sub('batch_size: [0-9]+',
               'batch_size: {}'.format(batch_size), s)

    # Set training steps, num_steps
    s = re.sub('num_steps: [0-9]+',
               'num_steps: {}'.format(num_steps), s)
    
    # Set number of classes num_classes.
    s = re.sub('num_classes: [0-9]+',
               'num_classes: {}'.format(num_classes), s)
    f.write(s)

In [33]:
!cat {pipeline_fname}

# SSD with Mobilenet v2 configuration for MSCOCO Dataset.
# Users should configure the fine_tune_checkpoint field in the train config as
# well as the label_map_path and input_path fields in the train_input_reader and
# eval_input_reader. Search for "PATH_TO_BE_CONFIGURED" to find the fields that
# should be configured.

model {
  ssd {
    num_classes: 35
    box_coder {
      faster_rcnn_box_coder {
        y_scale: 10.0
        x_scale: 10.0
        height_scale: 5.0
        width_scale: 5.0
      }
    }
    matcher {
      argmax_matcher {
        matched_threshold: 0.5
        unmatched_threshold: 0.5
        ignore_thresholds: false
        negatives_lower_than_unmatched: true
        force_match_for_each_row: true
      }
    }
    similarity_calculator {
      iou_similarity {
      }
    }
    anchor_generator {
      ssd_anchor_generator {
        num_layers: 6
        min_scale: 0.2
        max_scale: 0.95
        aspect_ratios: 1.0
        aspect_ratios: 2.0
        aspect

In [15]:
model_dir = 'training/'
# Optionally remove content in output model directory to fresh start.
!rm -rf {model_dir}
os.makedirs(model_dir, exist_ok=True)

model_dir

'training/'

#Run Tensorboard Optional

In [34]:
# !wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip -o ngrok-stable-linux-amd64.zip

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   


In [35]:
LOG_DIR = model_dir
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(LOG_DIR)
)

In [36]:
get_ipython().system_raw('./ngrok http 6006 &')

###Get Tensorboard link

In [37]:
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

https://09c7994059ac.ngrok.io


##Training the model

In [ ]:
# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=fine_tune_checkpoint,
                                                 save_weights_only=True,
                                                 verbose=1, period=5)

# Save the weights using the `checkpoint_path` format
#selected_model.save_weights(checkpoint_path.format(epoch=0))

# Loads the weights
selected_model.load_weights(fine_tune_checkpoint)


In [ ]:
#Training 

!python /content/fypdrive/models/research/object_detection/model_main.py \
    --pipeline_config_path={pipeline_fname} \
    --model_dir={model_dir} \
    --alsologtostderr \
    --num_train_steps={num_steps} \
    --num_eval_steps={num_eval_steps} \

In [24]:
!ls {model_dir}

ls: cannot access 'training/': No such file or directory


In [ ]:
# Legacy way of training(also works).
!python /content/fypdrive/models/research/object_detection/legacy/train.py --logtostderr --train_dir={model_dir} --pipeline_config_path={pipeline_fname}

Streaming output truncated to the last 5000 lines.
I0713 07:29:48.409713 140040688166656 supervisor.py:1050] Recording summary at step 1192660.
INFO:tensorflow:global step 1192661: loss = 4.4044 (0.533 sec/step)
I0713 07:29:48.572500 140044360394624 learning.py:512] global step 1192661: loss = 4.4044 (0.533 sec/step)
INFO:tensorflow:global step 1192662: loss = 4.2718 (0.408 sec/step)
I0713 07:29:48.993904 140044360394624 learning.py:512] global step 1192662: loss = 4.2718 (0.408 sec/step)
INFO:tensorflow:global step 1192663: loss = 4.0200 (0.362 sec/step)
I0713 07:29:49.380268 140044360394624 learning.py:512] global step 1192663: loss = 4.0200 (0.362 sec/step)
INFO:tensorflow:global step 1192664: loss = 4.3652 (0.491 sec/step)
I0713 07:29:49.883421 140044360394624 learning.py:512] global step 1192664: loss = 4.3652 (0.491 sec/step)
INFO:tensorflow:global step 1192665: loss = 4.3828 (0.541 sec/step)
I0713 07:29:50.577380 140044360394624 learning.py:512] global step 1192665: loss = 4.382

#To know the remaining time for google colab session

In [ ]:
import time, psutil
Start = time.time()- psutil.boot_time()
Left= 12*3600 - Start
print('Time remaining for this session is: ', Left/3600)

Time remaining for this session is:  2.729322369231118


##Saving the model for next training in Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [ ]:
!zip -r /content/file.zip /content/models/research/training


In [ ]:
from google.colab import files
files.download("/content/file.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
!zip -r /content/pretrained.zip /content/models/research/pretrained_model


In [ ]:
from google.colab import files
files.download("/content/pretrained.zip")

#For the purpose of continuing training after runtime restart (after <=12 hrs)

##Download each of `training/` files to google drive

In [ ]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once in a notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

#fname = os.path.basename(pb_fname)
# Create & upload a text file.
#events
uploaded = drive.CreateFile({'title': 'events.out.tfevents.1593241039.dba4504e9c5b'})
uploaded.SetContentFile('/content/models/research/training/events.out.tfevents.1593241039.dba4504e9c5b')
uploaded.Upload()

uploaded = drive.CreateFile({'title': 'events.out.tfevents.1593241213.dba4504e9c5b'})
uploaded.SetContentFile('/content/models/research/training/events.out.tfevents.1593241213.dba4504e9c5b')
uploaded.Upload()

uploaded = drive.CreateFile({'title': 'events.out.tfevents.1593265610.dba4504e9c5b'})
uploaded.SetContentFile('/content/models/research/training/events.out.tfevents.1593265610.dba4504e9c5b')
uploaded.Upload()

uploaded = drive.CreateFile({'title': 'events.out.tfevents.1593269394.dba4504e9c5b'})
uploaded.SetContentFile('/content/models/research/training/events.out.tfevents.1593269394.dba4504e9c5b')
uploaded.Upload()

uploaded = drive.CreateFile({'title': 'events.out.tfevents.1593273905.dba4504e9c5b'})
uploaded.SetContentFile('/content/models/research/training/events.out.tfevents.1593273905.dba4504e9c5b')
uploaded.Upload()

#graph
uploaded = drive.CreateFile({'title': 'graph.pbtxt'})
uploaded.SetContentFile('/content/models/research/training/graph.pbtxt')
uploaded.Upload()

#1
uploaded = drive.CreateFile({'title': 'model.ckpt-81742.data-00000-of-00001'})
uploaded.SetContentFile('/content/models/research/training/model.ckpt-81742.data-00000-of-00001')
uploaded.Upload()

uploaded = drive.CreateFile({'title': 'model.ckpt-81742.index'})
uploaded.SetContentFile('/content/models/research/training/model.ckpt-81742.index')
uploaded.Upload()

uploaded = drive.CreateFile({'title': 'model.ckpt-81742.meta'})
uploaded.SetContentFile('/content/models/research/training/model.ckpt-81742.meta')
uploaded.Upload()

#2
uploaded = drive.CreateFile({'title': 'model.ckpt-80390.data-00000-of-00001'})
uploaded.SetContentFile('/content/models/research/training/model.ckpt-80390.data-00000-of-00001')
uploaded.Upload()

uploaded = drive.CreateFile({'title': 'model.ckpt-80390.index'})
uploaded.SetContentFile('/content/models/research/training/model.ckpt-80390.index')
uploaded.Upload()

uploaded = drive.CreateFile({'title': 'model.ckpt-80390.meta'})
uploaded.SetContentFile('/content/models/research/training/model.ckpt-80390.meta')
uploaded.Upload()


#2
uploaded = drive.CreateFile({'title': 'model.ckpt-74859.data-00000-of-00001'})
uploaded.SetContentFile('/content/models/research/training/model.ckpt-74859.data-00000-of-00001')
uploaded.Upload()

uploaded = drive.CreateFile({'title': 'model.ckpt-74859.index'})
uploaded.SetContentFile('/content/models/research/training/model.ckpt-74859.index')
uploaded.Upload()

uploaded = drive.CreateFile({'title': 'model.ckpt-74859.meta'})
uploaded.SetContentFile('/content/models/research/training/model.ckpt-74859.meta')
uploaded.Upload()

#2
uploaded = drive.CreateFile({'title': 'model.ckpt-76273.data-00000-of-00001'})
uploaded.SetContentFile('/content/models/research/training/model.ckpt-76273.data-00000-of-00001')
uploaded.Upload()

uploaded = drive.CreateFile({'title': 'model.ckpt-76273.index'})
uploaded.SetContentFile('/content/models/research/training/model.ckpt-76273.index')
uploaded.Upload()

uploaded = drive.CreateFile({'title': 'model.ckpt-76273.meta'})
uploaded.SetContentFile('/content/models/research/training/model.ckpt-76273.meta')
uploaded.Upload()

#2
uploaded = drive.CreateFile({'title': 'model.ckpt-77711.data-00000-of-00001'})
uploaded.SetContentFile('/content/models/research/training/model.ckpt-77711.data-00000-of-00001')
uploaded.Upload()

uploaded = drive.CreateFile({'title': 'model.ckpt-77711.index'})
uploaded.SetContentFile('/content/models/research/training/model.ckpt-77711.index')
uploaded.Upload()

uploaded = drive.CreateFile({'title': 'model.ckpt-77711.meta'})
uploaded.SetContentFile('/content/models/research/training/model.ckpt-77711.meta')
uploaded.Upload()


#checkpoint
uploaded = drive.CreateFile({'title': 'checkpoint'})
uploaded.SetContentFile('/content/models/research/training/checkpoint')
uploaded.Upload()

#eval_0
uploaded = drive.CreateFile({'title': 'events.out.tfevents.1593242115.dba4504e9c5b'})
uploaded.SetContentFile('/content/models/research/training/eval_0/events.out.tfevents.1593242115.dba4504e9c5b')
uploaded.Upload()

uploaded = drive.CreateFile({'title': 'events.out.tfevents.1593266504.dba4504e9c5b'})
uploaded.SetContentFile('/content/models/research/training/eval_0/events.out.tfevents.1593266504.dba4504e9c5b')
uploaded.Upload()

uploaded = drive.CreateFile({'title': 'events.out.tfevents.1593270309.dba4504e9c5b'})
uploaded.SetContentFile('/content/models/research/training/eval_0/events.out.tfevents.1593270309.dba4504e9c5b')
uploaded.Upload()

uploaded = drive.CreateFile({'title': 'events.out.tfevents.1593274820.dba4504e9c5b'})
uploaded.SetContentFile('/content/models/research/training/eval_0/events.out.tfevents.1593274820.dba4504e9c5b')
uploaded.Upload()

print('Uploaded file with ID {}'.format(uploaded.get('id')))

##Download each of `pretrained_model/` to google drive

In [ ]:
uploaded = drive.CreateFile({'title': 'saved_model.pb'})
uploaded.SetContentFile('/content/models/research/pretrained_model/saved_model/saved_model.pb')
uploaded.Upload()

uploaded = drive.CreateFile({'title': 'checkpoint'})
uploaded.SetContentFile('/content/models/research/pretrained_model/checkpoint')
uploaded.Upload()

uploaded = drive.CreateFile({'title': 'model.ckpt.data-00000-of-00001'})
uploaded.SetContentFile('/content/models/research/pretrained_model/model.ckpt.data-00000-of-00001')
uploaded.Upload()

uploaded = drive.CreateFile({'title': 'model.ckpt.index'})
uploaded.SetContentFile('/content/models/research/pretrained_model/model.ckpt.index')
uploaded.Upload()

uploaded = drive.CreateFile({'title': 'model.ckpt.meta'})
uploaded.SetContentFile('/content/models/research/pretrained_model/model.ckpt.meta')
uploaded.Upload()

uploaded = drive.CreateFile({'title': 'frozen_inference_graph.pb'})
uploaded.SetContentFile('/content/models/research/pretrained_model/frozen_inference_graph.pb')
uploaded.Upload()


uploaded = drive.CreateFile({'title': 'pipeline.config'})
uploaded.SetContentFile('/content/models/research/pretrained_model/pipeline.config')
uploaded.Upload()

print('Uploaded file with ID {}'.format(uploaded.get('id')))

Uploaded file with ID 1oELSczIDam-_pF07kZSWK0V5A3Zxko5q


## Exporting a Trained Inference Graph
Once your training job is complete, you need to extract the newly trained inference graph, which will be later used to perform the object detection. This can be done as follows:

In [ ]:
import re
import numpy as np

output_directory = './fined_tuned_model'

lst = os.listdir(model_dir)
lst = [l for l in lst if 'model.ckpt-' in l and '.meta' in l]
steps=np.array([int(re.findall('\d+', l)[0]) for l in lst])
last_model = lst[steps.argmax()].replace('.meta', '')

last_model_path = os.path.join(model_dir, last_model)
print(last_model_path)
!python /content/fypdrive/models/research/object_detection/export_inference_graph.py \
    --input_type=image_tensor \
    --pipeline_config_path={pipeline_fname} \
    --output_directory={output_directory} \
    --trained_checkpoint_prefix={last_model_path}

FileNotFoundError: ignored

In [ ]:
!ls {output_directory}

ls: cannot access '{output_directory}': No such file or directory


## Download the model `.pb` file

In [ ]:
import os

pb_fname = os.path.join(os.path.abspath(output_directory), "frozen_inference_graph.pb")
assert os.path.isfile(pb_fname), '`{}` not exist'.format(pb_fname)

In [ ]:
!ls -alh {pb_fname}

### Option1 : upload the `.pb` file to your Google Drive
Then download it from your Google Drive to local file system.

During this step, you will be prompted to enter the token.

In [ ]:
# Install the PyDrive wrapper & import libraries.
# This only needs to be done once in a notebook.
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials


# Authenticate and create the PyDrive client.
# This only needs to be done once in a notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

fname = os.path.basename(pb_fname)
# Create & upload a text file.
uploaded = drive.CreateFile({'title': fname})
uploaded.SetContentFile(pb_fname)
uploaded.Upload()
print('Uploaded file with ID {}'.format(uploaded.get('id')))

### Option2 :  Download the `.pb` file directly to your local file system
This method may not be stable when downloading large files like the model `.pb` file. Try **option 1** instead if not working.

In [ ]:
from google.colab import files
files.download(pb_fname)

### Download the `label_map.pbtxt` file

In [ ]:
from google.colab import files
files.download(label_map_pbtxt_fname)

### Download the modified pipline file
If you plan to use OpenVINO toolkit to convert the `.pb` file to inference faster on Intel's hardware (CPU/GPU, Movidius, etc.)

In [ ]:
files.download(pipeline_fname)

In [ ]:
# !tar cfz fine_tuned_model.tar.gz fine_tuned_model
# from google.colab import files
# files.download('fine_tuned_model.tar.gz')

## Run inference test
Test with images in repository `object_detection_demo/test` directory.

In [ ]:
import re
import numpy as np

output_directory = './frozen_tflite'

In [ ]:
import os
import glob

# Path to frozen detection graph. This is the actual model that is used for the object detection.
PATH_TO_CKPT = pb_fname

# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = label_map_pbtxt_fname

# If you want to test the code with your images, just add images files to the PATH_TO_TEST_IMAGES_DIR.
PATH_TO_TEST_IMAGES_DIR =  os.path.join(repo_dir_path, "/data/images/test")

assert os.path.isfile(pb_fname)
assert os.path.isfile(PATH_TO_LABELS)
TEST_IMAGE_PATHS = glob.glob(os.path.join(PATH_TO_TEST_IMAGES_DIR, "*.*"))
assert len(TEST_IMAGE_PATHS) > 0, 'No image found in `{}`.'.format(PATH_TO_TEST_IMAGES_DIR)
print(TEST_IMAGE_PATHS)

In [ ]:
%cd /content/fypdrive/models/research/object_detection

import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile

from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image

# This is needed since the notebook is stored in the object_detection folder.
sys.path.append("..")
from object_detection.utils import ops as utils_ops


# This is needed to display the images.
%matplotlib inline


from object_detection.utils import label_map_util

from object_detection.utils import visualization_utils as vis_util


detection_graph = tf.Graph()
with detection_graph.as_default():
    od_graph_def = tf.GraphDef()
    with tf.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
        serialized_graph = fid.read()
        od_graph_def.ParseFromString(serialized_graph)
        tf.import_graph_def(od_graph_def, name='')


label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(
    label_map, max_num_classes=num_classes, use_display_name=True)
category_index = label_map_util.create_category_index(categories)


def load_image_into_numpy_array(image):
    (im_width, im_height) = image.size
    return np.array(image.getdata()).reshape(
        (im_height, im_width, 3)).astype(np.uint8)

# Size, in inches, of the output images.
IMAGE_SIZE = (12, 8)


def run_inference_for_single_image(image, graph):
    with graph.as_default():
        with tf.Session() as sess:
            # Get handles to input and output tensors
            ops = tf.get_default_graph().get_operations()
            all_tensor_names = {
                output.name for op in ops for output in op.outputs}
            tensor_dict = {}
            for key in [
                'num_detections', 'detection_boxes', 'detection_scores',
                'detection_classes', 'detection_masks'
            ]:
                tensor_name = key + ':0'
                if tensor_name in all_tensor_names:
                    tensor_dict[key] = tf.get_default_graph().get_tensor_by_name(
                        tensor_name)
            if 'detection_masks' in tensor_dict:
                # The following processing is only for single image
                detection_boxes = tf.squeeze(
                    tensor_dict['detection_boxes'], [0])
                detection_masks = tf.squeeze(
                    tensor_dict['detection_masks'], [0])
                # Reframe is required to translate mask from box coordinates to image coordinates and fit the image size.
                real_num_detection = tf.cast(
                    tensor_dict['num_detections'][0], tf.int32)
                detection_boxes = tf.slice(detection_boxes, [0, 0], [
                                           real_num_detection, -1])
                detection_masks = tf.slice(detection_masks, [0, 0, 0], [
                                           real_num_detection, -1, -1])
                detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
                    detection_masks, detection_boxes, image.shape[0], image.shape[1])
                detection_masks_reframed = tf.cast(
                    tf.greater(detection_masks_reframed, 0.5), tf.uint8)
                # Follow the convention by adding back the batch dimension
                tensor_dict['detection_masks'] = tf.expand_dims(
                    detection_masks_reframed, 0)
            image_tensor = tf.get_default_graph().get_tensor_by_name('image_tensor:0')

            # Run inference
            output_dict = sess.run(tensor_dict,
                                   feed_dict={image_tensor: np.expand_dims(image, 0)})

            # all outputs are float32 numpy arrays, so convert types as appropriate
            output_dict['num_detections'] = int(
                output_dict['num_detections'][0])
            output_dict['detection_classes'] = output_dict[
                'detection_classes'][0].astype(np.uint8)
            output_dict['detection_boxes'] = output_dict['detection_boxes'][0]
            output_dict['detection_scores'] = output_dict['detection_scores'][0]
            if 'detection_masks' in output_dict:
                output_dict['detection_masks'] = output_dict['detection_masks'][0]
    return output_dict


for image_path in TEST_IMAGE_PATHS:
    image = Image.open(image_path)
    # the array based representation of the image will be used later in order to prepare the
    # result image with boxes and labels on it.
    image_np = load_image_into_numpy_array(image)
    # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
    image_np_expanded = np.expand_dims(image_np, axis=0)
    # Actual detection.
    output_dict = run_inference_for_single_image(image_np, detection_graph)
    # Visualization of the results of a detection.
    vis_util.visualize_boxes_and_labels_on_image_array(
        image_np,
        output_dict['detection_boxes'],
        output_dict['detection_classes'],
        output_dict['detection_scores'],
        category_index,
        instance_masks=output_dict.get('detection_masks'),
        use_normalized_coordinates=True,
        line_thickness=8)
    plt.figure(figsize=IMAGE_SIZE)
    plt.imshow(image_np)

/content/drive/My Drive/models/research/object_detection


NameError: ignored

In [ ]:
!python /content/fypdrive/models/research/object_detection/export_tflite_ssd_graph.py --input_type image_tensor --pipeline_config_path /content/fypdrive/models/research/object_detection/samples/configs/ssd_mobilenet_v2_coco.config --trained_checkpoint_prefix /content/fypdrive/models/research/training/model.ckpt-815857 --output_directory /content/fypdrive/models/research/frozen_tflite/frozen_inference_graph.pb -add_postprocessing_op True --max_detections 10

In [ ]:
!tflite_convert --output_file=/content/fypdrive/models/research/frozen_tflite/detect.tflite --graph_def_file=/content/fypdrive/models/research/frozen_tflite/frozen_inference_graph.pb/tflite_graph.pb --input_arrays=normalized_input_image_tensor --output_arrays='TFLite_Detection_PostProcess','TFLite_Detection_PostProcess:1','TFLite_Detection_PostProcess:2','TFLite_Detection_PostProcess:3' --input_shape=1,300,300,3 --allow_custom_ops